In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision import transforms as T
from torchvision.transforms import ToTensor, Lambda
import numpy as np
from tqdm import tqdm
from torchvision.models import resnet18

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from pytorch_lightning.loggers import TensorBoardLogger
import matplotlib.pyplot as plt

from utils import LitModel
import argparse

import tensorboard


In [2]:

# argpaser
parser = argparse.ArgumentParser(description='PyTorch Lightning Example')
parser.add_argument('--batch_size', type=int, default=64)
parser.add_argument('--learning_rate', type=float, default=1e-3)
parser.add_argument('--epochs', type=int, default=100)
parser.add_argument('--num_tta', type=int, default=10)
parser.add_argument('--es_patience', type=int, default=10)
parser.add_argument('--num_workers', type=int, default=4)
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--progress_bar', type=bool, default=False)
parser.add_argument('--checkpoint_verbose', type=bool, default=False)
parser.add_argument('--earlystopping_verbose', type=bool, default=False)
args = parser.parse_args([]) # default settings

seed = args.seed


In [3]:
architecture = resnet18(pretrained=True)

In [4]:

augmentation = T.Compose([T.ToTensor(),
                                T.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
                                T.RandomHorizontalFlip(),
                                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

no_augmentation = T.Compose([T.ToTensor(),
                                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


In [5]:
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=augmentation
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=no_augmentation
)

test_data_tta = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=augmentation
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [6]:

train_dataset, val_dataset = random_split(training_data, [45000, 5000])

train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=False)
test_dataloader_tta = DataLoader(test_data_tta, batch_size=args.batch_size, num_workers=args.num_workers, shuffle=False)

logger = TensorBoardLogger("tb_logs", name="my_model")
early_stop_callback = EarlyStopping(monitor="val_loss", patience=args.es_patience, verbose=args.earlystopping_verbose, mode="min")
checkpoint_callback = ModelCheckpoint('models', save_top_k=1, monitor='val_loss', verbose=args.checkpoint_verbose, mode='min')

model = LitModel(architecture, args.learning_rate)
trainer = Trainer(max_epochs=args.epochs, 
                gpus=1,
                enable_progress_bar=args.progress_bar,
                logger=logger, 
                callbacks=[early_stop_callback, checkpoint_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
/home/kang/anaconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/kang/pytorch_lightning_pipeline/models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/home/kang/anaconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:662: UserWarning: Your `val_dataloader` has `shuffle=True`, it is strongly recommended that you turn this off for val/test/predict dataloaders.
  category=UserWarning,


epoch:  0 [train_loss: 1.2603 val_loss: 1.1226] [train_acc: 0.5652 val_acc: 0.6232]
epoch:  1 [train_loss: 0.9849 val_loss: 0.9376] [train_acc: 0.6636 val_acc: 0.6832]
epoch:  2 [train_loss: 0.8765 val_loss: 0.8520] [train_acc: 0.6982 val_acc: 0.7066]
epoch:  3 [train_loss: 0.8304 val_loss: 0.8452] [train_acc: 0.7139 val_acc: 0.7008]
epoch:  4 [train_loss: 0.7667 val_loss: 0.9281] [train_acc: 0.7371 val_acc: 0.6906]
epoch:  5 [train_loss: 0.7856 val_loss: 0.7251] [train_acc: 0.7302 val_acc: 0.7554]
epoch:  6 [train_loss: 0.7161 val_loss: 0.7642] [train_acc: 0.7546 val_acc: 0.7436]
epoch:  7 [train_loss: 0.6845 val_loss: 0.7487] [train_acc: 0.7619 val_acc: 0.7424]
epoch:  8 [train_loss: 0.6563 val_loss: 0.7084] [train_acc: 0.7743 val_acc: 0.7588]
epoch:  9 [train_loss: 0.6362 val_loss: 0.8137] [train_acc: 0.7818 val_acc: 0.7238]
epoch: 10 [train_loss: 0.6158 val_loss: 0.6904] [train_acc: 0.7854 val_acc: 0.7592]
epoch: 11 [train_loss: 0.6574 val_loss: 0.7328] [train_acc: 0.7766 val_acc: 

In [8]:
trainer.test(test_dataloaders=test_dataloader)

/home/kang/anaconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:907: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  "`trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6."
/home/kang/anaconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1399: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /home/kang/pytorch_lightning_pipeline/models/epoch=27-step=19711.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /h

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8515999913215637, 'test_loss': 0.4361374080181122}
--------------------------------------------------------------------------------


[{'test_loss': 0.4361374080181122, 'test_acc': 0.8515999913215637}]

In [9]:
tta_pred_list = []
for _ in tqdm(range(args.num_tta)):
    y_hat = torch.vstack(trainer.predict(model=model, dataloaders=test_dataloader_tta))
    tta_pred_list.append(y_hat)
tta_pred_mean = torch.stack(tta_pred_list).mean(0)

tta_acc = np.mean(tta_pred_mean.argmax(1).numpy() == np.array(test_data.targets))
print(f"TTA accuracy: {tta_acc}")

100%|██████████| 10/10 [00:13<00:00,  1.39s/it]

TTA accuracy: 0.8604
